<a href="https://colab.research.google.com/github/VinithaJ88/Coursera_Capstone/blob/main/SegmentingandClusteringNeighbourhoodsinToronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


# Converting Wikipedia Page on Toronto into a pandas dataframe




In [ ]:
pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [ ]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
torontotable=soup.find('table',{'class':"wikitable"})


In [ ]:
df=pd.read_html(str(torontotable))
# convert list to dataframe
df=pd.DataFrame(df[0])
#removing the rows where Borough is not assigned
df = df[df.Borough != 'Not assigned']
#identify rows where postal code is duplicated to fulfil the instruction - More than one neighborhood can exist in one postal code area.
duplicateRowsDF = df[df.duplicated(['Postal Code'])] 
print("Duplicate Rows based on Postal Code are:", duplicateRowsDF, sep='\n')
#identify the rows where Neighborhood is 'not assigned' to fulfil the instruction - If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df2=df.loc[df.Neighbourhood == 'Not assigned']
print(df2)
#print the shape of the remaining dataframe
print(df.shape)

Duplicate Rows based on Postal Code are:
Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []
Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []
(103, 3)


In [47]:
from google.colab import files
uploaded = files.upload()



Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [48]:
import io
import pandas as pd
df3 = pd.read_csv(io.BytesIO(uploaded['Geospatial_Coordinates.csv']))
print(df3.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [50]:
#print new dataframe with latitudes and longitudes
df4=pd.merge(df, df3, on='Postal Code')
print(df4.tail())

    Postal Code           Borough  ...   Latitude  Longitude
98          M8X         Etobicoke  ...  43.653654 -79.506944
99          M4Y  Downtown Toronto  ...  43.665860 -79.383160
100         M7Y      East Toronto  ...  43.662744 -79.321558
101         M8Y         Etobicoke  ...  43.636258 -79.498509
102         M8Z         Etobicoke  ...  43.628841 -79.520999

[5 rows x 5 columns]


In [51]:
#Visualise neighborhoods using Folium

import folium
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [59]:
from sklearn.cluster import KMeans
import numpy as np
import random

k=5
toronto_clustering = df4.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Clust Labels', kmeans.labels_)
df4

,Clust Labels,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...,...
98,2,1,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,0,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,1,4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,2,1,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [61]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Clust Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

